<a href="https://colab.research.google.com/github/evmpython/CAT212_2025/blob/main/CAT212_Aula_06_Mapa_relampagos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# **Disciplina:** CAT212 - Microfísica de Nuvens e Eletricidade Atmosférica
---

## 🟢 **Aula Prática 6:** $\underline{Criando\ Mapas\ de\ Relâmpagos}$

---


**OBJETIVO:**
 - Nesta aula prática aprenderemos como criar mapas da densidade de relâmpagos (relâmpagos/km$^{2}$). Para isto utilizaremos os dados de relâmpagos da rede [Earth Networks Total Lightning Network (ENTLN)](https://www.earthnetworks.com/why-us/networks/lightning/). Para mais informações sobre a ENTLN clique [aqui](https://www.earthnetworks.com/Portals/0/pdf/ENTLN%20Global%20Slick_v4).



---



**DADOS DE ENTRADA**:

Arquivos texto de relâmpagos.

- **Tipo do dado:** tabela  
- **Formato do dado:** arquivo TEXTO
- **Nome do arquivo:** pulse_CLIMATE_2020-01-01.txt.gz
- **Fonte dos dados:** Rede EarthNetwork, cedido para fins didáticos pela CLIMATEMPO



---

**DADOS DE SAÍDA:**
- **Tipo do dado:** Figuras, arquivos netcdf e tabelas
- **Formato do dado:** arquivo PNG, NETCDF e TXT
- **Imagens geradas:**
    1. Fig_1_relampagos_mes.png
    2. Fig_2_anual.png
    3. Fig_3_mensal.jpg
    4. raios_2018-01-01.nc
    5. raios.txt



---

**PROCEDIMENTO REALIZADO:**
 - Nessa aula realizaremos os seguintes procedimentos:

    1.   Instalação das bibliotecas
    2.   Importação das bibliotecas
    3.   Montando drive
    4.   Definindo o caminho do diretório
    5.   Declarando funções
    6.   Carregando shapefiles
    7.   Destargeando os arquivos
    8.   Informações da matriz de raios
    9.   Plotar mapa para um determinado mês
    10.  Pré-tratamento dos dados
    11.  Plotar mapa de acumulado de relâmpagos para um ano
    12.  Plotar mapa de acumulado mensal de relâmpagos para um ano
    13.  Salvar os dados em arquivo ASCII

  

---



**OBSERVAÇÕES IMPORTANTES**:
1. Mudar o caminho para o seu drive.
2. Escolha o Estado para qual será produzido o mapa.

---
**PROFESSOR:**

 - Enrique Vieira Mattos - UNIFEI: enrique@unifei.edu.br / https://github.com/evmpython
 - Atualizado em: 11/04/2025


---


# **1° Passo:** Instalando as bibliotecas

In [ ]:
!pip install -q ultraplot cartopy salem rasterio

# **2° Passo:** Importando as bibliotecas

In [ ]:
import numpy as np                          # Biblioteca para trabalhar com matrizes
import pandas as pd                         # Biblioteca para trabalhar com tabelas (parecidas com tabelas em excel)
import ultraplot as uplt                    # Biblioteca para gerar figuras
import os                                   # Biblioteca para acessar o sistema operacional
import cartopy.crs as ccrs                  # Biblioteca para trabalhar com projeções de mapas
import cartopy.io.shapereader as shpreader  # Biblioteca para ler arquivos shapefiles
import tarfile                              # Biblioteca para destargear um arquivo
from tqdm.notebook import tqdm_notebook     # Biblioteca que cira barra de progresso
import salem                                # Biblioteca para mascarar regiões
import xarray as xr                         # Biblioteca para para ler/criar arquivos netcdf
import glob                                 # Biblioteca para para listar arquivos
import calendar                             # Biblioteca para calendário
import time                                 # Biblioteca de contagem de tempo

# vamos ignorar vários avisos
import warnings
warnings.filterwarnings('ignore')

# **3° Passo:** Montando drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **4° Passo:** Caminho do diretório - **### >>> MUDANÇAS AQUI <<< ###**

- Insira aqui o caminho da pasta do seu Google Drive onde esta o código dessa aula: `alterar aqui`

In [ ]:
dir = '/content/drive/MyDrive/00_DISPONIBILIZAÇÃO_DE_AULAS/1_PRIMEIRO_SEMESTRE/CAT212 - MICROFÍSICA DE NUVENS E ELETRICIDADE ATMOSFÉRICA/AULAS/CAT212 - PYTHON PARA MICROFISICA DE NUVENS/AULA_06_RELAMPAGOS_MAPAS'

- **Criando a pasta de saída ("output")**

In [ ]:
os.makedirs(f'{dir}/output/', exist_ok=True)
os.makedirs(f'/content/txt/', exist_ok=True)
os.makedirs(f'/content/netcdf/', exist_ok=True)

# **5° Passo:** Declara função

##**Função que calcula o índice i e j da localização do relâmpago**

In [ ]:
# Função que calcula o índice i e j da localização do relâmpago
def index(longitudes_matriz, latitudes_matriz, lon_raio, lat_raio):

    ''' Função para calcular o índice (i e j) do pixel de uma matriz que o relâmpago pertence

    Parâmetros:
               longitudes_matriz (array): array de uma dimensão das longitudes da matriz em graus
               latitudes_matriz (array): array de uma dimensão das latitudes da matriz em graus
               lon_raio (float): valor da longitude do relâmpago em graus
               lat_raio (float): valor da latitude do relâmpago em graus

    Retorna:
            indice_lat_raio (float): índice da latitude (ou seja, da linha) do pixel da matriz que o relâmpago pertence
            indice_lon_raio (float): índice da longitude (ou seja, da coluna) do pixel da matriz que o relâmpago pertence
    '''

    # calcula a diferença entre as lats/lons da matriz e a latitude/longitude do relâmpago
    distancia_lon = (longitudes_matriz - lon_raio)**2
    distancia_lat = (latitudes_matriz - lat_raio)**2

    # índice da longitude e latitude do relâmpago
    indice_lon_raio = np.nonzero(distancia_lon == np.min(distancia_lon))
    indice_lat_raio = np.nonzero(distancia_lat == np.min(distancia_lat))

    # retorna os valores dos índices calculados
    return indice_lat_raio, indice_lon_raio

# **6° Passo:** Shapefiles - **### >>> MUDANÇAS AQUI <<< ###**


- Iremos utilizar os arquivos shapefiles do território brasileiro. Os dados serão baixados direto do meu [GitHub](https://github.com/evmpython/shapefile).

In [ ]:
#--------------------------------------------------------------#
#                  SHAPEFILES DO ESTADO
#--------------------------------------------------------------#
# sigla do Estado
# escolha o Estado. Lembre de escolher apenas entre os Estados da região Sudeste: SP, RJ, MG, ES
sigla_estado = 'MG'

# url do github
url =  'https://github.com/evmpython/shapefile/raw/main/UFs/'

# nomes completos dos arquivos
file1 = f'{url}{sigla_estado}/{sigla_estado}_UF_2019.cpg'
file2 = f'{url}{sigla_estado}/{sigla_estado}_UF_2019.dbf'
file3 = f'{url}{sigla_estado}/{sigla_estado}_UF_2019.prj'
file4 = f'{url}{sigla_estado}/{sigla_estado}_UF_2019.shp'
file5 = f'{url}{sigla_estado}/{sigla_estado}_UF_2019.shx'

# baixa o shapefile
!wget -c {file1} # 'https://github.com/evmpython/shapefile/raw/main/UFs/AC/AC_UF_2019.shp
!wget -c {file2}
!wget -c {file3}
!wget -c {file4}
!wget -c {file5}

# lendo shapefile de todos Estados do Brasil
estado = salem.read_shapefile(f'/content/{sigla_estado}_UF_2019.shp')

# designa os limites do Estado
lonmin, lonmax, latmin, latmax = float(estado['min_x']), float(estado['max_x']), float(estado['min_y']), float(estado['max_y'])

# exibe os limites do shapefile
print(lonmin, lonmax, latmin, latmax )

#**PARTE 1:** Destargeando o arquivo de relâmpagos


- São arquivos de relâmpagos para o ano de 2018, separados por dia. Após destargeados eles serão salvos no Drive na pasta `/content/txt`.

In [ ]:
%%time
# Caminho onde o arquivo .tar.gz está localizado
input_path = f'{dir}/input/pulse_CLIMATE_2018.tar.gz'

# Caminho onde os arquivos extraídos serão salvos
output_path = f'/content/txt'

# Abrir o arquivo tar.gz
tar = tarfile.open(input_path)

# Listar os nomes dos arquivos dentro do tar
tar.getnames()

# Extrair todos os arquivos para o caminho desejado
tar.extractall(path=output_path)

# Fechar o arquivo tar
tar.close()

#**PARTE 2:** Informações da matriz de raios

In [ ]:
# Configuração da grade
#lonmin = -51.3
#lonmax = -39.7
#latmin = -23.0
#latmax = -14.0

# Espaçamento da grade
delta = 0.01   # grade com 1 km de resolução espacial

# Montando a grade
lons = np.arange(lonmin, lonmax, delta)
lats = np.arange(latmax, latmin, -delta)

# Quantidade de pontos para longitude e latitude
nlon = len(lons)
nlat = len(lats)

In [ ]:
lats

In [ ]:
lons

In [ ]:
nlon

In [ ]:
nlat

#**PARTE 3:** Mapa para um determinado mês

**Selecionando o mês de janeiro**

In [ ]:
# mês escolhido
mes = 1

# nome por extenso dos meses
nome_meses = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho',
              'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

# monta lista dos arquivos daquele mês
files = sorted(glob.glob(f'/content/txt/pulse_CLIMATE_2018-{str(mes).zfill(2)}*.txt.gz')) # /content/txt/pulse_CLIMATE_2018-01-01.txt.gz

# mostra a lista de arquivos
files

**Leitura do dado**

In [ ]:
%%time
# Cria uma lista vazia
df_list = []

# Loop dos arquivos por dia
for file in tqdm_notebook(files):

    print('PROCESSANDO === >>>:', file)

    # Leitura do arquivo
    df0 = pd.read_csv(file, delimiter='\s+', compression='gzip', usecols=['tipo','ano','mes','dia','hor','min','seg','lat','lon','pc(A)'])

    # Seleciona os relâmpagos dentro do Estado escolhido
    df1 = df0[ (df0['lon'] > lonmin) & (df0['lon'] < lonmax) & (df0['lat'] > latmin) & (df0['lat'] < latmax)]

    # Insere a tabela numa lista
    df_list.append(df1)

# Junta todas as tabelas em uma única tabela
df = pd.concat(df_list, ignore_index=True)

**Mostrando o dado**

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

**Calculando os raios por ponto de grade**

In [ ]:
%%time
# Transforma de series para numpy.array
raios_lon, raios_lat = df['lon'].values, df['lat'].values

In [ ]:
# Declara a matriz de relâmpagos
raios = np.zeros((nlat, nlon))  # Exemplo: raios = np.zeros(qte_linha, qte_coluna)

In [ ]:
%%time
# Loop em cada longitude e latitude da lista
for lonraio, latraio in zip(raios_lon, raios_lat):

    # função que extrai a qual pixel (ou seja, determina as variáveis 'lin' e 'col') aquele relâmpago pertence
    lin, col = index(lons, lats, lonraio, latraio)

    # soma os relâmpagos por pixel
    raios[lin,col]+=1

    """
    Exemplo:
      raios[2, 4] = 1
      raios[2, 4] = 2 (1+1)
      raios[2, 4] = 3 (2+1)
    """

**Transforma a matriz para Xarray.DataSet**

In [ ]:
data_vars = {'raio':(('lat', 'lon'), raios, {'units': 'ocorrências/km²', 'long_name':'Raios'})}
coords = {'lat': lats, 'lon': lons}
ds = xr.Dataset(data_vars=data_vars, coords=coords)

In [ ]:
ds

In [ ]:
ds['raio'].plot()

**Plota figura**

In [ ]:
%%time
# moldura da figura
fig, ax = uplt.subplots(figsize=(7,5), tight=True, proj='pcarree')

# fomatação dos eixos da figura
ax.format(coast=False, borders=False, innerborders=False,
          labels=True, latlines=5, lonlines=5,
          latlim=(latmin-0.08, latmax+0.08), lonlim=(lonmin-0.08, lonmax+0.08),
          title=f'Mapa de Relâmpagos - {nome_meses[mes-1]}',
          small='15px', large='20px', abc=False)

# substituir zeros por NaN na variável 'raio'
ds['raio'] = ds['raio'].where(ds['raio'] != 0, np.nan)

# plota figura
map1 = ax.contourf(ds['lon'],
                   ds['lat'],
                   ds['raio'].salem.roi(shape=estado),
                   cmap='jet',
                   vmin=0,
                   vmax=60,
                   levels=uplt.arange(0, 40, 5),
                   extend='max')

# adiciona shapefile do Estado
shapefile=list(shpreader.Reader(f'/content/{sigla_estado}_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black',facecolor='none', linewidth=0.7)

# plota barra de cores da figura
fig.colorbar(map1, loc='r', label='Relâmpagos/km$^{2}$', length=0.97, ticklabelsize=13, labelsize=15, space=0.3)

# salva figura
fig.save(f'{dir}/output/Fig_1_relampagos_mes_{nome_meses[mes-1]}_{sigla_estado}.png', dpi=300)

# exibe na tela
uplt.show()

#**PARTE 4:** Pré-Tratamento dos dados para a `Parte 5, 6 e 7`

**Gera arquivos Netcdf por dia**.
- Esta etapa demora 11 min para processar.

In [ ]:
%%time
# Loop dos arquivos por dia
for file in tqdm_notebook(sorted(glob.glob('/content/txt/pulse*.txt.gz'))):

    # imprime na tela o nome do arquivo
    print('PROCESSANDO === >>>:', file)

    # Leitura do arquivo com o Pandas
    df0 = pd.read_csv(file, delimiter='\s+', compression='gzip', usecols=['tipo','ano','mes','dia','hor','min','seg','lat','lon','pc(A)'])

    # Seleciona os relâmpagos dentro do Estado de MG
    tabela2 = df0[ (df0['lon'] > lonmin) & (df0['lon'] < lonmax) & (df0['lat'] > latmin) & (df0['lat'] < latmax)]

    # Gera matriz de raios
    raios_lon, raios_lat = tabela2['lon'].values, tabela2['lat'].values

    # Interpolando para ponto de grade
    raios = np.zeros((nlat, nlon))

    # Loop em cada longitude e latitude da lista
    for lonraio, latraio in zip(raios_lon, raios_lat):

        # função que extrai a qual pixel aquele relâmpago pertence
        lin, col = index(lons, lats, lonraio, latraio)

        # soma os relâmpagos por pixel
        raios[lin,col]+=1

    # Gera arquivo netcdf
    data_vars = {'raio':(('lat', 'lon'), raios, {'units': 'ocorrências/km²', 'long_name':'Raios'})}
    coords = {'lat': lats, 'lon': lons, 'time': pd.to_datetime(f'{file[27:37]}')}
    ds = xr.Dataset(data_vars=data_vars, coords=coords)
    ds.to_netcdf(f'/content/netcdf/raios_{file[27:37]}.nc')

**Leitura dos arquivos netcdfs**.


In [ ]:
dados = xr.open_mfdataset(f'/content/netcdf/raios*.nc', concat_dim='time', combine='nested')
dados

**Agrupamento dos dados por Ano e por Mês**

In [ ]:
# ANUAL
anual = dados.resample(time='1Y').sum() # ou anual = dados.groupby('time.year').sum('time')
anual

In [ ]:
# MENSAL
mensal = dados.resample(time='1M').sum() # ou mensal = dados.groupby('time.month').sum('time')
mensal

#**PARTE 5:** Mapa Anual

**Mostrando a variável**

In [ ]:
anual

**Plota a figura**

In [ ]:
# moldura da figura
fig, ax = uplt.subplots(figsize=(7,5), tight=True, proj='pcarree')

# fomatação dos eixos da figura
ax.format(coast=False, borders=False, innerborders=False,
          labels=True, latlines=5, lonlines=5,
          latlim=(latmin-0.08, latmax+0.08), lonlim=(lonmin-0.08, lonmax+0.08),
          title=f'Mapa de Relâmpagos para 2018',
          small='15px', large='20px', abc=False)

# substituir zeros por NaN na variável 'raio'
anual['raio'] = anual['raio'].where(anual['raio'] != 0, np.nan)

# plota figura
map1 = ax.contourf(anual['lon'],
                   anual['lat'],
                   anual['raio'][0,:,:].salem.roi(shape=estado),
                   cmap='jet',
                   vmin=0,
                   vmax=60,
                   levels=uplt.arange(0, 60, 5),
                   extend='max')

# adiciona shapeFile de MG
shapefile=list(shpreader.Reader(f'/content/{sigla_estado}_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black',facecolor='none', linewidth=0.7)

# plota barra de cores da figura
fig.colorbar(map1, loc='r', label='Relâmpagos/ano.km$^{2}$', length=0.98, ticklabelsize=13, labelsize=15, space=0.17)

# salva figura
fig.save(f'{dir}/output/Fig_2_relampagos_total_anual_{sigla_estado}.png', dpi=300)

# exibe na tela
uplt.show()

#**PARTE 6:** Mapa mensal

**Mostrando a variável**

In [ ]:
mensal

**Plotando a figura**

In [ ]:
# cria a moldura da figura
fig, ax = uplt.subplots(axwidth=3, axheight=3, ncols=3, nrows=4, tight=True, proj='pcarree')

# formatação dos eixos
ax.format(coast=False, borders=False, innerborders=False,
          labels=True, latlines=4, lonlines=4,
          latlim=(latmin-0.08, latmax+0.08), lonlim=(lonmin-0.08, lonmax+0.08),
          suptitle=f'Mapa de Relâmpagos para 2018',
          small='20px', large='25px',
          abc=True, abcstyle='a)', abcsize=20)

# adiciona shapefile do estado
shapefile=list(shpreader.Reader(f'/content/{sigla_estado}_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black',facecolor='none', linewidth=0.7)

# substituir zeros por NaN na variável 'raio'
mensal['raio'] = mensal['raio'].where(mensal['raio'] != 0, np.nan)

# loop dos meses
for imes in range(0, 12):

  print(f'Processando o mês: .... {imes}')

  # aplica mascara recortando o Estado no arquivo
  xr_shp = mensal['raio'][imes,:,:].salem.roi(shape=estado)

  # plota figura
  map1 = ax[imes].contourf(mensal['lon'],
                           mensal['lat'],
                           xr_shp,
                           cmap='jet',
                           vmin=0,
                           vmax=20,
                           levels=uplt.arange(0, 20, 2),
                           extend='max')

        #ax[0]...
        #ax[1]...
        #ax[12]...

  # plota titulo de cada figura
  ax[imes].format(title=nome_meses[imes-1])

# plota barra de cores da figura
fig.colorbar(map1, loc='b', label='Relâmpagos/mês.km$^{2}$', length=0.99, ticklabelsize=13, labelsize=15)

# salva figura
fig.save(f'{dir}/output/Fig_3_relampagos_total_mensal_{sigla_estado}.jpg', dpi=300)

# exibe na tela
uplt.show()

#**PARTE 7:** Salvando a quantidade de relâmpagos mensal para um **arquivo ASCII**

**Contabiliza os relâmpagos para cada mês**

In [ ]:
nome_mes = []
raios_mes = []

# Loop dos meses
for imes in range(0, 12):

    # nome do mês
    mes = calendar.month_name[int(imes+1)]

    # junta o nome dos meses numa lista
    nome_mes.append(mes)

    print('Processando --->>>', mes)

    # contabiliza os relâmpagos dentro do Estado escolhido
    raios = float(mensal['raio'][imes,:,:].salem.roi(shape=estado).sum(dim=('lon','lat')).values)

    #raios = 200
    #raios = 500

    # junta o valor da quantidade de relâmpagos numa lista
    raios_mes.append(raios)

    # raios_mes.append(200, 500...)

**Salva num arquivo**

In [ ]:
with open(f'{dir}/output/raios_{sigla_estado}.txt', 'w') as arquivo:
    for imes, iraios in zip(nome_mes, raios_mes):
        arquivo.write(f'{imes} {iraios} \n')
        print(f'{imes} {iraios}')